<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [11]:
import glob
from google.colab import drive
drive.mount('/content/drive')
##folder lists
ls = !ls "/content/drive/My Drive/words/sample"
ls = str(ls).strip("'[")
ls = ls.strip("]'")
entries = ls.split("\\t")
entries

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['month1', 'month2', 'month3']

In [45]:
## preparation for text preprocessing
from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english'))) + ["us", "also", "said", "would", "could", "mr", "get"]
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop_words

['this',
 'you',
 'haven',
 'needn',
 'been',
 'does',
 'no',
 'not',
 'm',
 'we',
 'again',
 "weren't",
 'hadn',
 'of',
 'in',
 'from',
 'the',
 'own',
 'before',
 'while',
 'to',
 'once',
 'when',
 'some',
 'or',
 'them',
 "shouldn't",
 "you've",
 'd',
 "don't",
 'yours',
 'are',
 'aren',
 "isn't",
 'was',
 'whom',
 "should've",
 'themselves',
 'other',
 'its',
 'both',
 've',
 "didn't",
 'myself',
 "doesn't",
 "mustn't",
 "mightn't",
 'off',
 'because',
 'so',
 "aren't",
 'isn',
 'his',
 'as',
 'couldn',
 'after',
 'above',
 "you'll",
 'an',
 "needn't",
 'few',
 'out',
 'ma',
 'during',
 're',
 'those',
 'my',
 "wouldn't",
 'had',
 'too',
 'himself',
 's',
 'but',
 'very',
 'ourselves',
 't',
 'me',
 'there',
 'weren',
 'i',
 'only',
 'hers',
 'between',
 'through',
 'itself',
 'into',
 'down',
 'y',
 'ours',
 "shan't",
 'with',
 "you'd",
 "that'll",
 'just',
 'shouldn',
 'up',
 'our',
 'their',
 'has',
 'mightn',
 'can',
 'yourself',
 'he',
 'who',
 'here',
 'they',
 'she',
 'am',


In [33]:
import re
def removePunctuation(text):
    punctuation = '!,;:?"\'.-'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip().lower()
 

In [46]:
#loading corpus and preprocessing

corpus_list= []
for entry in entries:
    filepath = "/".join(["/content/drive/My Drive/words/sample", entry, "*.txt"])
    data_files = glob.glob(filepath)
    for data_file in data_files:
      document = open(data_file,'r').read().lower()
      text = [word for word in document.split()] 
      corpus = [removePunctuation(word) for word in text]
      corpus = [word for word in corpus if not word in stop_words]
      corpus_list += [corpus]
len(corpus_list)





1323

hello world


In [48]:
#3 Merge corpus into one document
with open("/content/drive/My Drive/words/sample/BleiCorpus.txt", 'w') as File:
  for corpus in corpus_list:
    File.write(" ".join(corpus)+ "\n" )

In [42]:
## establish dictionary 
dictionary = corpora.Dictionary(corpus_list)
dictionary.save("/content/drive/My Drive/words/sample/word_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/sample/BleiCorpus.txt"):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()
corpus = [vector for vector in corpus_memory_friendly]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/sample/news_corpus', corpus)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [43]:
## loading and setting
try:
  dictionary = Dictionary.load("/content/drive/My Drive/words/sample/word_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")

corpus = bleicorpus.BleiCorpus("/content/drive/My Drive/words/sample/news_corpus")

time_slice = [438, 429, 456]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [44]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5)

KeyboardInterrupt: ignored

In [18]:
#Printing Topics
ldaseq.print_topics(time=0)

[[('-', 0.008138278623003488),
  ('best', 0.00722372065121603),
  ('"i', 0.003934130706098536),
  ('said', 0.0036999887694496108),
  ('also', 0.0036959581622512064),
  ('would', 0.003585114741708679),
  ('game', 0.0034908308346543954),
  ('last', 0.0032523035753691384),
  ('one', 0.0031271346991542407),
  ('first', 0.003118835596116297),
  ('two', 0.002962258656993341),
  ('play', 0.002930826166947557),
  ('chelsea', 0.0028835498994196204),
  ('win', 0.0027817720829047764),
  ('players', 0.0027509820025130847),
  ('club', 0.002691209454365762),
  ('"we', 0.0026885415940103136),
  ('united', 0.00267593149993234),
  ('league', 0.002576014943323031),
  ('think', 0.0025063518687069982)],
 [('said', 0.011476252321617105),
  ('-', 0.007431551340182253),
  ('us', 0.007130325291501011),
  ('also', 0.004444234242588271),
  ('new', 0.004319346995075073),
  ('last', 0.003648447774472637),
  ('would', 0.0035846050470246535),
  ('could', 0.0035544552450907235),
  ('year', 0.00344925406338575),
  ('

In [ ]:
#Topic Evolution
ldaseq.print_topic_times(topic=0)

In [ ]:
## word distribution in a specific document
words = [dictionary[word_id] for word_id, count in corpus[558]]
print (words)

In [ ]:
## A specific document's latent topic distribution
doc = ldaseq.doc_topics(558)
print(doc)

In [ ]:
doc_football_1 = ['economy', 'bank', 'mobile', 'phone', 'markets', 'buy', 'football', 'united', 'giggs'] #make up a news about how football affect economy
doc_football_1 = dictionary.doc2bow(doc_football_1)
doc_football_1 = ldaseq[doc_football_1]
print (doc_football_1)

In [ ]:
## topic distance between documents (hellinger distance: measure similarity of two distribution)
doc_football_2 = ['arsenal', 'fourth', 'wenger', 'oil', 'middle', 'east', 'sanction', 'fluctuation']
doc_football_2 = dictionary.doc2bow(doc_football_2)
doc_football_2 = ldaseq[doc_football_2]
hellinger(doc_football_1, doc_football_2)

In [ ]:
## changing rate of topic factors(words in topics)
ldaseq.print_topic_times(1)

##Note: the chain_variance parameter could affect the result